# Price regression

In [1]:
import sklearn as sk
import pandas as pd

In [69]:
df = pd.read_csv('../data/data.csv')
df = df.drop(columns=['id', 'date', 'condition'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21613 non-null  float64
 1   bedrooms       21613 non-null  int64  
 2   bathrooms      21613 non-null  float64
 3   sqft_living    21613 non-null  int64  
 4   sqft_lot       21613 non-null  int64  
 5   floors         21613 non-null  float64
 6   waterfront     21613 non-null  object 
 7   view           21613 non-null  int64  
 8   grade          21613 non-null  int64  
 9   sqft_above     21613 non-null  int64  
 10  sqft_basement  21613 non-null  int64  
 11  yr_built       21613 non-null  int64  
 12  yr_renovated   21613 non-null  int64  
 13  zipcode        21613 non-null  int64  
 14  lat            21613 non-null  float64
 15  long           21613 non-null  float64
 16  sqft_living15  21613 non-null  int64  
 17  sqft_lot15     21613 non-null  int64  
dtypes: flo

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,N,0,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,N,0,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,N,0,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,N,0,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,N,0,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## train/test split

In [70]:
X = pd.get_dummies(df.drop(columns=["price"]))
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(X, df['price'], test_size=0.2)

i'm going to use 3 different models

In [75]:
linelar = sk.linear_model.LinearRegression()
randf = sk.ensemble.RandomForestRegressor()
grad = sk.ensemble.GradientBoostingRegressor()

In [76]:
linelar.fit(x_train, y_train)
randf.fit(x_train, y_train)
grad.fit(x_train, y_train)

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [77]:
p1 = linelar.predict(x_test)
p2 = randf.predict(x_test)
p3 = grad.predict(x_test)

In [79]:
e1 = sk.metrics.mean_absolute_error(y_test, p1)
e2 = sk.metrics.mean_absolute_error(y_test, p2)
e3 = sk.metrics.mean_absolute_error(y_test, p3)

print(e1)
print(e2)
print(e3)

128974.04573064421
72789.91594972077
80948.87678398969


so gradient boost is the best.

let's try catboost

In [80]:
from catboost import CatBoostRegressor

In [81]:
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(df.drop(columns=["price"]), df['price'], test_size=0.2)

In [82]:
catb = CatBoostRegressor()

cat_cols = x_train.select_dtypes(include=['object']).columns
catb.fit(x_train, y_train, cat_features=list(cat_cols))
p4 = catb.predict(x_test)
e4 = sk.metrics.mean_absolute_error(y_test, p4)
print(e4)

Learning rate set to 0.064232
0:	learn: 358198.5580393	total: 3.14ms	remaining: 3.13s
1:	learn: 343855.2772652	total: 4.95ms	remaining: 2.47s
2:	learn: 330415.7081045	total: 7.34ms	remaining: 2.44s
3:	learn: 318322.5588549	total: 9.07ms	remaining: 2.26s
4:	learn: 306941.5764580	total: 11.2ms	remaining: 2.23s
5:	learn: 296811.0125745	total: 12.9ms	remaining: 2.14s
6:	learn: 287469.8161663	total: 15ms	remaining: 2.13s
7:	learn: 278464.9370690	total: 16.8ms	remaining: 2.09s
8:	learn: 270086.7041106	total: 18.9ms	remaining: 2.08s
9:	learn: 262064.7717723	total: 20.9ms	remaining: 2.07s
10:	learn: 254182.5977392	total: 22.9ms	remaining: 2.06s
11:	learn: 246607.2920535	total: 24.9ms	remaining: 2.05s
12:	learn: 239611.9823522	total: 26.9ms	remaining: 2.04s
13:	learn: 232634.2328171	total: 28.7ms	remaining: 2.02s
14:	learn: 226689.3613908	total: 30.8ms	remaining: 2.02s
15:	learn: 221169.7589423	total: 32.8ms	remaining: 2.02s
16:	learn: 216339.8918813	total: 35.1ms	remaining: 2.03s
17:	learn: 21

it's more better. 